In [2]:
%load_ext nvcc4jupyter

from nvcc4jupyter import set_defaults
set_defaults(compiler_args='-arch=sm_100a -Xptxas=-v -O0 -I/workspace/cutlass/include')

Source files will be saved in "/tmp/tmpwohn4s39".


In [5]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda.h>
#include <cuda_fp16.h>
#include <stdint.h>
#include <cuda_bf16.h>

#define allocate_tmem(smem_dst, nCols) \ 
  asm volatile("tcgen05.alloc.cta_group::1.sync.aligned{.shared::cta}.b32  [%0], %1"); \
    :                                                                              \ 
    : "r"(smem_dst), "r"(nCols) \
    : "memory");\
               
#define deallocate_tmem(taddr, nCols) \ 
  asm volatile("tcgen05.dealloc.cta_group::1.sync.aligned.b32    [%0], %1"); \
    :                                                                        \ 
    : "r"(taddr), "r"(nCols) \
    : "memory");\
               
constexpr int N_cols = 32; 

__global__ void alloc_dealloc_tmem()
{
  __shared__ uint32_t A[1]; 
  uint32_t smem_addr = (uint32_t) &A[0];
  uint32_t n_cols = N_cols;
  allocate_tmem(smem_addr, n_cols); 
  uint32_t taddr = A[0]; 
  deallocate_tmem(taddr, n_cols);
  
}


int main()
{
  alloc_dealloc_tmem<<<1,32>>>();
  cudaDeviceSynchronize();
  return 0;
}

/tmp/tmpwohn4s39/4b175c7f-9c06-43d9-b968-704326085277/single_file.cu:10:40: warning: backslash and newline separated by space
   10 | #define allocate_tmem(smem_dst, nCols) \
      |                                         
/tmp/tmpwohn4s39/4b175c7f-9c06-43d9-b968-704326085277/single_file.cu:12:84: warning: backslash and newline separated by space
   12 |     :                                                                              \
      |                                                                                     
/tmp/tmpwohn4s39/4b175c7f-9c06-43d9-b968-704326085277/single_file.cu:16:39: warning: backslash and newline separated by space
   16 | #define deallocate_tmem(taddr, nCols) \
      |                                        
/tmp/tmpwohn4s39/4b175c7f-9c06-43d9-b968-704326085277/single_file.cu:18:78: warning: backslash and newline separated by space
   18 |     :                                                                        \
      |                      

Ok this is so fucked and I am so pissed off right now. 

In [14]:
l = 37 
c = 53 
def get_tmem_addr(lane, col): 
  return (c | (l<< 16))

def get_2d_from_TMEM_addr(tmem_addr): 
  c_mask = (1 << 16) - 1 
  col = tmem_addr & c_mask 
  row = tmem_addr >> 16
  return row, col
  
l_new, c_new  = get_2d_from_TMEM_addr(get_tmem_addr(l,c))
print(l_new, c_new)

tmem_addr = get_tmem_addr(l,c)
print(f"{tmem_addr:0{32}b}")

def print_bin(n): 
  print(f"{n:0{32}b}")

37 53
00000000001001010000000000110101


In [15]:
print_bin(tmem_addr % 2**16)

00000000000000000000000000110101


In [16]:
print_bin(tmem_addr // (2**16))

00000000000000000000000000100101


In [10]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda.h>
#include <cuda_fp16.h>
#include <stdint.h>
#include <cuda_bf16.h>


#define allocate_tmem(smem_dst, nCols) \
  asm volatile("tcgen05.alloc.cta_group::1.sync.aligned.shared::cta.b32  [%0], %1" \
    : /* no outputs */ \
    : "r"(smem_dst), "r"(nCols) \
    : "memory")

// FIX 1: Removed trailing whitespace and internal semicolons
// FIX 3: Changed constraint for taddr (now taddr_ptr) from "r" to "l"
#define deallocate_tmem(taddr_ptr, nCols) \
  asm volatile("tcgen05.dealloc.cta_group::1.sync.aligned.b32    %0, %1" \
    : /* no outputs */ \
    : "r"(taddr_ptr), "r"(nCols) \
    : "memory")

constexpr int N_cols = 32;

__global__ void alloc_dealloc_tmem()
{
  __shared__ uint32_t A[1];
  

  uint32_t smem_addr = (uint32_t) &A[0];
  uint32_t n_cols = N_cols;
  
  allocate_tmem(smem_addr, n_cols);

  uint32_t taddr = A[0];
  

  deallocate_tmem(taddr, n_cols);
}


int main()
{
  alloc_dealloc_tmem<<<1,32>>>();
  cudaDeviceSynchronize();
  printf("Kernel executed successfully.\n"); // Added a print for confirmation
  return 0;
}

/tmp/tmpwohn4s39/b4d84de6-380d-44ca-8576-33a101d68ddd/single_file.cu(32): warning #767-D: conversion from pointer to smaller integer
    uint32_t smem_addr = (uint32_t) &A[0];
                         ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/tmp/tmpwohn4s39/b4d84de6-380d-44ca-8576-33a101d68ddd/single_file.cu(32): warning #767-D: conversion from pointer to smaller integer
    uint32_t smem_addr = (uint32_t) &A[0];
                         ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/tmp/tmpwohn4s39/b4d84de6-380d-44ca-8576-33a101d68ddd/single_file.cu(32): warning #767-D: conversion from pointer to smaller integer
    uint32_t smem_addr = (uint32_t) &A[0];
                         ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

ptxas /tmp/tmpxft_00000858_00000000-6_single_file.compute_100a.ptx, line 31; fatal   : Parsing error near 'ld': syntax error
ptxas fatal   : Ptx assemb

In [13]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda.h>
#include <cuda_fp16.h>
#include <stdint.h>
#include <cuda_bf16.h>

// CORRECT: Uses a 64-bit pointer "l" constraint and [memory] operand
#define allocate_tmem(smem_dst_ptr, nCols) \
  asm volatile("tcgen05.alloc.cta_group::1.sync.aligned.shared::cta.b32  [%0], %1;" \
    : /* no outputs */ \
    : "l"(smem_dst_ptr), "r"(nCols) \
    : "memory")

// CORRECTED: Uses a 32-bit "r" constraint for the handle value
// and NO BRACKETS [] on the operand %0.
#define deallocate_tmem(taddr_handle, nCols) \
  asm volatile("tcgen05.dealloc.cta_group::1.sync.aligned.b32    %0, %1;" \
    : /* no outputs */ \
    : "r"(taddr_handle), "r"(nCols) \
    : "memory")

constexpr int N_cols = 32;

__global__ void alloc_dealloc_tmem()
{
  __shared__ uint32_t A[1]; 
  
  // 1. Get 64-bit POINTER to the shared memory location
  uint64_t smem_addr = (uint64_t) &A[0];
  uint32_t n_cols = N_cols;
  
  // 2. Allocate: Pass the POINTER. The handle is written to A[0].
  allocate_tmem(smem_addr, n_cols); 
  
  // 3. Load the 32-bit HANDLE VALUE from shared memory
  uint32_t taddr = A[0]; 
  
  // 4. Deallocate: Pass the HANDLE VALUE.
  deallocate_tmem(taddr, n_cols);
}


int main()
{
  alloc_dealloc_tmem<<<1,32>>>();
  cudaDeviceSynchronize();
  printf("Kernel executed successfully.\n");
  return 0;
}

Kernel executed successfully.

